# RedTideProcessing

> Fill in a module description here

In [ ]:
#| default_exp RedTideProcessing

In [ ]:
#| hide
from nbdev.showdoc import *
import nbdev

In [ ]:
#| hide
nbdev.nbdev_export()

In [ ]:
#| export
import glob
import numpy             as np
import pandas            as pd
import cv2               as cv
import datetime          as dt
import glob
import matplotlib.pylab  as plt
import numpy             as np
import pandas            as pd
import pandas            as pd
import time
from geographiclib.geodesic import Geodesic
from           PIL          import Image   
from       skimage          import io

In [ ]:
## from google.colab.patches   import cv2_imshow # for image display

#| hide
# Testing.

#| hide
## Testing includes

In [ ]:
import panel as pn
from bokeh.plotting import figure, show 
from bokeh.io import show, output_notebook

In [ ]:
output_notebook()

Loading BokehJS ...

#| hide
## Global vars for testing only.

For working with test data from s:

In [ ]:
Gps_File_Name = '/mnt/f/My Drive/Missions/2021-1208-HAB-Lake-Parker/'\
                '2021-1208/2021-1208-175324-910850-mission-gps.txt'

mission_folder = '/mnt/s/2021-0717-HAB-Spec-Tampa-Bay-PIE/2021-0717-HAB-pie/'

Gps_File_Name = "/mnt/s/2021-0717-HAB-Spec-Tampa-Bay-PIE/2021-0717-HAB-pie/2021-0717-044645-687230-mission-gps.txt"

test_spectra_file = mission_folder+'/160206/hab_spectra/2021-0717-160401-066388-spec.json'

In [ ]:
mission_folder = "../sample_data/2023-0718/034901/"
test_spectra_file = mission_folder + "/hab_spectra/" + "2023-0718-034912-259843-spec.json"
test_spectra_file

'../sample_data/2023-0718/034901//hab_spectra/2023-0718-034912-259843-spec.json'

#| hide
## Module wide Globals

#| hide
### Working spectra data 

In [ ]:
#| export
summed_rows = 0
Xpixels     = np.array(1)
wavelengths = np.array(1)
create_time = dt.datetime.utcnow()

In [ ]:
class HAB_Spectra:
  '''
  '''
  Xpixels     = np.array(1)
  wavelengths = np.array(1)
  create_time = dt.datetime.utcnow()
  pixel_0      = None
  wavelength_0 = None
  pixel_1      = None
  wavelength_1 = None
  def __init__(self):
    self.lat          = None
    self.lon          = None
    self.altitde_m    = None
    self.n_saturated  = None
    self.raw_y_min    = None
    
    self.raw_y        = []
    self.summed_rows  = 0
  
  def __str__(self):
    str = f'{self.lat=} {self.lon=} {self.altitde_m=} {self.summed_rows=} {self.n_saturated=} {self.raw_y_min=}'
    return str


In [ ]:
hab_spectra = HAB_Spectra()
hab_spectra.__str__()

'self.lat=None self.lon=None self.altitde_m=None self.summed_rows=0 self.n_saturated=None self.raw_y_min=None'

#| hide
## Functions

#| hide#
### General Purpose datetime Functions

#| hide
#### def hhmmss_to_sod( hhmmss, Usecs=0):

In [ ]:
#| export
def hhmmss_to_sod( hhmmss, Usecs=0):
    """
    Converts a time string in 'HHMMSS' format to a seconds-of-the-day
    including an optional microseconds fraction.

    Parameters:
    ----------- 
    hhmmss : str
    Usecs  : str, default=0

    Returns: 
    --------
    float
      Seconds of the day including the fractional part computed from `Usecs`
    
    Description:
    ------------
    hhmmss where hh is hours, mm is minutes, and ss is seconds. Example '123456'
    is 12 hours, 34 minutes, and 56 seconds. A string representing the number of
    microseconds since the last second increment, ie the fractional part of a second. 
    Example: '954561' represents 0.954561 seconds, or 954,561 microseconds since the
    last seconds rollover. Returns A single floating point value of the seconds since
    midnight plus the fractional seconds.

    Examples:
    ---------
    To be added.

    """
    hh = int(hhmmss[0:2])
    mm = int(hhmmss[2:4])
    ss = int(hhmmss[4:6])
    fsod = hh*3600 + mm*60 + ss + float(Usecs) * 1e-6
    return fsod

#| hide
#### def now_utc( fmt='%Y-%m%d %H:%M:%S'):

In [ ]:
#| export
def now_utc( fmt='%Y-%m%d %H:%M:%S'):
    """ Return the current UTC date and time as a string.  Example return: '2021-0812 15:39:41'.
    See: http://shorturl.at/koOQ7 for timeformat options & directives.
    """
    t = dt.datetime.utcnow()
    ts = f'{t:{fmt}}'
    return ts

#| hide
#### def hms2sod( str, debug=False):

In [ ]:
#| export
#debug_hms2sod = False
def hms2sod( str, debug=False):
  '''
  Converts an ASCII string in the form 'HH:MM"SS' to seconds of the day. Works with
  fractional seconds.
  '''
  t_lst = str.split('.')
  ts = time.strptime( t_lst[0], '%H:%M:%S')
  if len(t_lst) > 1:
    fs = float("."+t_lst[1] )
  else:
    fs = 0.0
  sod = ts[3]*3600 + ts[4]*60 + ts[5] + fs
  return sod

In [ ]:
print('\nTesting: hms2sod( t )')
tt = ['00:00:00',   '23:59:59', '12:00:00',
      '00:00:00.0', '0:0:0.0', '0:0:0' ]
print('Time Str       Seconds of Day\n'\
       '-----------------------------')
for t in tt:
  rv = hms2sod( t )
  print(f'{t:16} {rv:12.6f}')


Testing: hms2sod( t )
Time Str       Seconds of Day
-----------------------------
00:00:00             0.000000
23:59:59         86399.000000
12:00:00         43200.000000
00:00:00.0           0.000000
0:0:0.0              0.000000
0:0:0                0.000000


|! hide
####   def extract_hms( fn):

In [ ]:
#| export
def extract_hms( fn):
    '''Extract the HHMMSS and Fsecs strings from filename "fn"

    Returns:
    --------
    ( hhmmss:str, fsecs:str, hhmmss.fsecs:float, sod.fsecs:float )
    '''
    lst = fn.split('-')
    sod = hhmmss_to_sod( lst[-3], Usecs=lst[-2] )
    return ( lst[-3], lst[-2], float(lst[-3])+float(lst[-2])*1e-6, sod  )

#| hide
### HABSpec JSON Functions

#| hide
#### def configure_json_spectra()

In [ ]:
#| export
def configure_json_spectra(f:str   # Json settings file.
                        ) ->str:   # Returns the Json file name.
    '''
    Configures HabSpec internal setting according to settings found within a json spectra file 'f'.

    Parameters:
    -----------
    f : str
        HabSpec json spectra full path and file name.

    Returns:
    --------
        f:str.
    '''
    global Xpixels, summed_rows, wavelengths 
    spec = pd.read_json(f)                      # get a spectra fron a json file
    y = np.array(spec['hab_spec'].spectra )     # Load the y values to an np array
    summed_rows = spec['hab_spec'].summed_rows
    Xpixels = np.arange(0,y.size )              # Construct an array of X values
    wavelengths = np.arange( y.size )
    return f

In [ ]:
fn = '../sample_data/2023-0718/034901/hab_spectra/2023-0718-034901-714246-spec.json'
f = configure_json_spectra( fn )
f, Xpixels, summed_rows, wavelengths

('../sample_data/2023-0718/034901/hab_spectra/2023-0718-034901-714246-spec.json',
 array([   0,    1,    2, ..., 1277, 1278, 1279]),
 800,
 array([   0,    1,    2, ..., 1277, 1278, 1279]))

|! hide
#### def calibrate_using_2_wavelengths()

In [ ]:
#| export
def calibrate_using_2_wavelengths(
                                  pixel0:int=0,         #  pixel # for Wavelength 0
                                  wavelength0:float=0,   # Wavelength of pixel 0
                                  pixel1:int=0,         # pixel # for Wavelength 1
                                  wavelength1:float=0   # Wavelength for pixel1
                                 ):
    '''
    Generate a Numpy array of calibration wavelenghts for each pixel. configure_json_spectra(f)
    must be called beforehand inrder to set the correct number of pixels.

    Parameters:
    -----------
    pixel0 : int, default 0
    wavelength0 : float, default 0
    pixel1 : int, default 0
    wavelength1 : float, default 0

    Returns:
    --------
    Float array of calibrated wavelengths for each pixel.  The returned array is the
    same size as Xpixels.

    Example:
    ---------
    # Configure the spectra by reading a spectra file.
    fn = '/content/drive/MyDrive/Missions/2021-0717-HAB-pie/165347/hab_spectra/2021-0717-165348-272814-spec.json'
    hs.configure_json_spectra(fn)

    # Calibrate the spectra.  Pixel 73 is at 430.774nm, and pixel 941 is at 759.370nm
    hs.calibrate_using_2_wavelengths(pixel0=73, wavelength0=430.774, pixel1=941, wavelength1=759.370 )

    References:
    -----------
    The following are wavelength calibration sources.
    See: http://hyperphysics.phy-astr.gsu.edu/hbase/quantum/atspect2.html
    and: https://commons.wikimedia.org/wiki/File:Fluorescent_lighting_spectrum_peaks_labelled.gif
    and: https://en.wikipedia.org/wiki/Fluorescent_lamp#/media/File:Spectra-Philips_32T8_natural_sunshine_fluorescent_light.svg
    '''
    dp = pixel1      - pixel0              # pixel delta
    dw = wavelength1 - wavelength0         # Wavelength delta
    slope = dw/dp                          # Linear slope
    wavelengths = wavelength0 + (Xpixels-pixel0) * slope
    return wavelengths

Calibrate spectra by setting the pixel numbers and cooresponding wavelengths
for two pixels.  The two pixels should be a far apart as possible to minimize
error.  The calibrate_using_2_wavelengths() function will return a list 
which contains the wavelength for every pixel in the spectra.

In [ ]:
# Calibrate the spectra for wavelength in nanometers
wavelengths = calibrate_using_2_wavelengths( 
  pixel0 =  73, wavelength0 = 430.774, 
  pixel1 = 941, wavelength1 = 759.370 )

print('\nWavelength Calibration Data:\n\n'
'                     First      Second\n'
f'   Wavelengths (nm): {wavelengths[73]:7.3f}      {wavelengths[941]:7.3f}\n'
f'     Pixel Location: {73:7.3f}      {941:7.3f}\n')

print(f'{len( wavelengths )} elements found in `wavelengths`.' )
print(f'First 5 pixel wavelengths: {wavelengths[0:5]}')
print(f' Last 5 pixel wavelengths: {wavelengths[-5:]}')


Wavelength Calibration Data:

                     First      Second
   Wavelengths (nm): 430.774      759.370
     Pixel Location:  73.000      941.000

1280 elements found in `wavelengths`.
First 5 pixel wavelengths: [403.13862212 403.51718894 403.89575576 404.27432258 404.6528894 ]
 Last 5 pixel wavelengths: [885.81131797 886.18988479 886.56845161 886.94701843 887.32558525]


|! hide
#### def read_spectra()

In [ ]:
#| export
def read_spectra(f,                    # JSON Spectra file name
                   remove_bias = True, # Bol remove bias
                   y_average   = True  # Average Y values
                  ) -> float:          # Numpy array of spectral points.
    """
    Reads a Json hyperspectra file.

    Parameters:
    -----------
    f : str
      The full path name of a hyperspectra Json file.
    remove_bias : Default True
      Subtracts the minimum y value from the array of y values to remove dark current. 
      The minimum value will typically be found in the IR side and from the pixel 
      sensors that are optically obscured.
    y_average : Default True
      The Json values are the sum total of each pixel column. Each column contains 800
      pixels. Setting this to True causes the read y values to be divided by 800.

    Returns:
    --------
    Numpy array
      An array of numpy float values representing the intensity values at each pixel. 

    Description:
    ------------
    Reads a Json spectra from a file.

    Examples:
    ---------
    fn = '/content/drive/MyDrive/Missions/2021-0717-HAB-pie/165347/hab_spectra/2021-0717-165348-272814-spec.json'
    s = hs.read_spectra(fn)
    """
    spec = pd.read_json(f)
    if summed_rows == 0 :
      configure_json_spectra(f)
    y = np.array(spec['hab_spec'].spectra)

    if y_average == True:
      y = y / summed_rows

    if remove_bias == True:
      y = y - y.min()
    return y

In [ ]:
print(test_spectra_file)
test_spectra = read_spectra(test_spectra_file)

../sample_data/2023-0718/034901//hab_spectra/2023-0718-034912-259843-spec.json


#| hide
#### def w_to_pixel(x, w))

In [ ]:
#| export
def w_to_pixel( 
  x,       #   Array of wavelengths for each pixel
  w        #   The wavelength you want the pixel number of.
) -> int:  #   The pixel
  '''
  Return the pixel index for a given wavelength `w`.  Out of range
  wavelengths cause -1 to be returned.
  '''
  dw  = (x[-1] - x[0])
  dpx = len(x)
  d  = dw/dpx
  px = int((w-x[0]) / d )
  if px < 0: 
    px = -1
  elif w > x[-1]:
    px = -1
  return px

#| hide
#### def pixel_to_w()

In [ ]:
#| export
def pixel_to_w(
  x,             # Array of wavlengths if the sensor.
  pix,           # Pixel number to convert to wavelength.
) -> float:      # The wavelength of the pixel.
  '''
  Return the wavelength of a given pixel index.
  Out of range wavelengths cause -1 to be returned.
  '''
  dw  = (x[-1] - x[0])
  dpx = len(x)
  d  = dw/dpx
  w = px*d + x[0]
  if w < x[0]:
    w = -1.0
  elif w > x[-1]:
    w = -1.0
  return w

test w_to_pixel.

In [ ]:
print('       w0   px        w1      Diff')
for w0 in ( 
  wavelengths[0], 
  300, 500, 600, 700, 800, 1000,
  wavelengths[-1] 
):
  px = w_to_pixel( wavelengths, w0)
  w1  = pixel_to_w( wavelengths, px)
  dif = w1 - w0
  print(f'{w0:9.3f} {px:4d} {w1:9.3f} {dif:9.3f}' )

       w0   px        w1      Diff
  403.139    0   403.139     0.000
  300.000   -1    -1.000  -301.000
  500.000  256   499.976    -0.024
  600.000  520   599.840    -0.160
  700.000  784   699.703    -0.297
  800.000 1049   799.945    -0.055
 1000.000   -1    -1.000 -1001.000
  887.326 1280   887.326     0.000


|! hide
#### def w_range_pixels( x,a,b):

In [ ]:
#| export
def w_range_pixels( 
  x,        # A numpy array of wavlengths for each pixel.
  a,        # Starting wavelength.
  b         # Ending wavelength.
) -> list:  # List of pixels cooresponding to `a`:`b` wavelengths.
    '''
    Returns a pixel index list of all pixels between wavelength 'a' and 'b'. 
    '''
    rv = np.where(np.logical_and( x>a, x<b) )
    return rv

Test w_range_pixels() between 500 and 530 nanometers.

In [ ]:
test_blue_band_edge = 500
test_red_band_edge  = 520
test_band_pixels = w_range_pixels( wavelengths, test_blue_band_edge, test_red_band_edge)

A numpy array is returned containing the pixel locations for each wavelength
between `test_blue_band_edge` and `test_red_band_edge`

In [ ]:
display(test_band_pixels)

(array([256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268,
        269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281,
        282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294,
        295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307,
        308]),)

We can compute numpy statistic on the spectra values within this band.  We will compute the mean, average, and
min and max on the test_spectra.

In [ ]:
( test_spectra[test_band_pixels].mean(),
  np.average( test_spectra[test_band_pixels] ), 
  test_spectra[test_band_pixels].min(), 
  test_spectra[test_band_pixels].max() 
)

(5.2126179245283035, 5.2126179245283035, 3.978749999999998, 6.438749999999999)

#| hide
####   def get_list_of_json_spectra( p):

In [ ]:
#| export
def get_list_of_json_spectra( p        # String path to mission data set.
                            ) ->list:  # List of full paths to each JSON spectra file.
    ''' Returns a list of Json spectra full path filenames found in subdirs under "p". '''
    gstr = f'{p}/*/*/*-spec.json'
    lst = glob.glob(gstr, recursive=True)
    return lst

Test and demo get_list_of_json_spectra()

In [ ]:
mission_path = '../sample_data/2023-0718'
mission_path = '/mnt/s/2021-0717-HAB-Spec-Tampa-Bay-PIE/2021-0717-HAB-pie/'
test_spectra_list = get_list_of_json_spectra( mission_path )
print(f'{len(test_spectra_list):,} JSON spectra files found.')
display("First 3", test_spectra_list[0:3])
display("Last 3",test_spectra_list[-3:])

11,273 JSON spectra files found.


'First 3'

['/mnt/s/2021-0717-HAB-Spec-Tampa-Bay-PIE/2021-0717-HAB-pie/141625/hab_spectra/2021-0717-141626-175824-spec.json',
 '/mnt/s/2021-0717-HAB-Spec-Tampa-Bay-PIE/2021-0717-HAB-pie/141625/hab_spectra/2021-0717-141626-882736-spec.json',
 '/mnt/s/2021-0717-HAB-Spec-Tampa-Bay-PIE/2021-0717-HAB-pie/141625/hab_spectra/2021-0717-141627-601038-spec.json']

'Last 3'

['/mnt/s/2021-0717-HAB-Spec-Tampa-Bay-PIE/2021-0717-HAB-pie/165347/hab_spectra/2021-0717-170224-525748-spec.json',
 '/mnt/s/2021-0717-HAB-Spec-Tampa-Bay-PIE/2021-0717-HAB-pie/165347/hab_spectra/2021-0717-170225-225927-spec.json',
 '/mnt/s/2021-0717-HAB-Spec-Tampa-Bay-PIE/2021-0717-HAB-pie/165347/hab_spectra/2021-0717-170225-920240-spec.json']

#| hide
####   def json_specs_to_df( specs ):

In [ ]:
#| export
def json_specs_to_df( specs ):
    '''
    Extracts the hhmmss and fsecs from 'specs' filename, converts
    the hhmmss and fsecs strings to float SOD.fsecs.

    Parameters:
    specs: list  A list of json spectra files.

    Returns:
    --------
    A DataFrame of sod.fsecs:float, hhmmss:str, Json_spec:str
    '''
    # Extract the hhmmss and fsecs from each file name,
    # convert the hhmmss and fsecs to float SOD.fsecs, 
    # and create a list of each.
    hhmmss = []
    fsecs  = []
    sod    = []
    for v in specs:
      tx = extract_hms( v )
      hhmmss.append( tx[0] )   # Build the list if hhmmss strings.
      sod.append( tx[3] )      # Build the list of sod floats.

    # Now convert the  lists to a Pandas dataframe.
    dct = {'sod':sod, 'hhmmss':hhmmss, 'Json_spec':specs}
    df = pd.DataFrame( dct )
    df.sort_values(by=['sod'], inplace=True)              # Sort the data in time order.
    return df

Test configuring, calibrating, and loading a JSON spectra file.

In [ ]:
print(f'{dt.datetime.utcnow() }: Testing creating HyperSpectral.')

# Use a json spectra to initialize things.
print(f'{dt.datetime.utcnow() }: Loading a json Spectra file to calibrate wavelength.')

test_spectra_file = '../sample_data/2023-0718/034901/hab_spectra/2023-0718-034902-840971-spec.json'
configure_json_spectra(test_spectra_file)

# Calibrate the spectra for wavelength in nanometers
print(f'{dt.datetime.utcnow() }: Calibrating wavelength.')
wavelengths = calibrate_using_2_wavelengths( 
  pixel0 =  73, wavelength0 = 430.774, 
  pixel1 = 941, wavelength1 = 759.370 )

print('\nWavelength Calibration Data:\n\n'
'                     First      Second\n'
f'   Wavelengths (nm): {wavelengths[73]:7.3f}      {wavelengths[941]:7.3f}\n'
f'     Pixel Location: {73:7.3f}      {941:7.3f}\n')

# Read a spectra array from a file into s
print(f'{dt.datetime.utcnow() }: Reading a spectra into "s"')
spectra = read_spectra( test_spectra_file )

# Plot the spectra vs pixel number.
#plt.plot(s)

2023-07-23 15:26:23.718742: Testing creating HyperSpectral.
2023-07-23 15:26:23.718830: Loading a json Spectra file to calibrate wavelength.
2023-07-23 15:26:23.722660: Calibrating wavelength.

Wavelength Calibration Data:

                     First      Second
   Wavelengths (nm): 430.774      759.370
     Pixel Location:  73.000      941.000

2023-07-23 15:26:23.722860: Reading a spectra into "s"


#### def is_water( x,y, debug=False):

In [ ]:
#| export
def is_water( x,y):
    '''
    Returns the mean signal value between 840nm and 860nm wavelengths.  Since water absorbs IR.

    Inputs:
      x   A numpy array of wavlengths for each pixel
      y   A numpy array of intensity values at each wavelength.  x and y mus be the same size.

    Returns:
      The mean signal value between 840nm and 860nm.  The signal level is not currently normalized
      for anything, exposure, etc.  Threshold is around 4.0.  Above 4, land or glint.

    References: 
    Application of the water-related spectral reflectance indices: A review
    https://www.sciencedirect.com/science/article/abs/pii/S1470160X18308215

    '''
    rv = y[w_range_pixels(x,840,860)].mean()
    return rv

#| hide
#### def wavelength_2_RGB()

In [ ]:
#| export
def wavelength_2_RGB(
  wavelength,   # Wavelength in nanometers.
  alpha=255     # Alpha to use.
) -> list:      # ( red, green, blue )
  '''
  Convert a wavelength to a tuple of red, green, blue values.
  
  From: https://codingmess.blogspot.com/2009/05/conversion-of-wavelength-in-nanometers.html
  '''
  w = int(wavelength)

  # colour
  if w >= 380 and w < 440:
      R = -(w - 440.) / (440. - 350.)
      G = 0.0
      B = 1.0
  elif w >= 440 and w < 490:
      R = 0.0
      G = (w - 440.) / (490. - 440.)
      B = 1.0
  elif w >= 490 and w < 510:
      R = 0.0
      G = 1.0
      B = -(w - 510.) / (510. - 490.)
  elif w >= 510 and w < 580:
      R = (w - 510.) / (580. - 510.)
      G = 1.0
      B = 0.0
  elif w >= 580 and w < 645:
      R = 1.0
      G = -(w - 645.) / (645. - 580.)
      B = 0.0
  elif w >= 645 and w <= 780:
      R = 1.0
      G = 0.0
      B = 0.0
  else:
      R = 0.0
      G = 0.0
      B = 0.0

  # intensity correction
  if w >= 380 and w < 420:
      SSS = 0.3 + 0.7*(w - 350) / (420 - 350)
  elif w >= 420 and w <= 700:
      SSS = 1.0
  elif w > 700 and w <= 780:
      SSS = 0.3 + 0.7*(780 - w) / (780 - 700)
  else:
      SSS = 0.0
  SSS *= 255

  return [int(SSS*R), int(SSS*G), int(SSS*B), int(alpha)]

In [ ]:
w = 500
alpha = 255
rgb = wavelength_2_RGB( w, alpha=alpha )
print(f'{w=}    {rgb=}  {alpha=} ')

w=500    rgb=[0, 255, 127, 255]  alpha=255 


#| hide
#### def wavelength_2_RGB_str()

In [ ]:
#| export
def wavelength_2_RGB_str( 
  wavelength:float,        # Wavelength in nanometers.
  alpha=255                # Alpha, transparency to use.  
) ->str:                   # rgb string such as: "#01abffff"
  '''
  Converts a wavelength to an RGB string suitable to use
  as a color parameter in most applications.
  '''
  rgb  = wavelength_2_RGB(wavelength, alpha=alpha)
  rgbs = f'#{rgb[0]:02x}{rgb[1]:02x}{rgb[2]:02x}{rgb[3]:02x}'
  return rgbs

In [ ]:
w = 500
alpha = 127
rgbs = wavelength_2_RGB_str( w, alpha=alpha )
print(f'{w=}    {rgbs=} ')

w=500    rgbs='#00ff7f7f' 


#| hide
### Fraunhofer_lines

In [ ]:
#| export
Fraunhofer_lines = {
                'A' :[ 'O2', 759.370  ],
                'B' :[ 'O2', 686.719  ], 
                'C' :[ 'Ha', 656.281  ],
                'a' :[ 'O2', 627.661  ],
                'D1':[ 'Na', 589.592  ],
                'D2':[ 'Na', 588.995  ],
                'D3':[ 'He', 587.5618 ],
              'e-hg':[ 'Hg', 546.073  ],
                'E2':[ 'Fe', 527.039  ],
                'b1':[ 'Mg', 518.362  ],
                'b2':[ 'Mg', 517.270  ],
                'b3':[ 'Mg', 516.891  ],
                'b4':[ 'Mg', 516.733  ],
                'c' :[ 'Fe', 495.761  ],                                                            
                'F' :[ 'Hb', 486.134  ],
                'd' :[ 'Fe', 466.814  ],
              'e-Fe':[ 'Fe', 438.355  ],
                'G' :[ 'Fe', 430.790  ],
                'G2':[ 'Ca', 430.774  ],
                'H' :[ 'Ca', 396.847  ] 
}

In [ ]:
Fraunhofer_lines['D1']

['Na', 589.592]

In [ ]:
Fraunhofer_lines['D1'][0]

'Na'

#| hide
### HG Lines

In [ ]:
#| export
HG_lines = { 
          'Hg-404' :[ 'Hg', 404.6563 ],
          'Hg-436' :[ 'Hg', 435.8328 ],
          'Hg-543' :[ 'Hg', 543.6    ],
          'Hg-546' :[ 'Hg', 546.0735 ],
          'Hg-576' :[ 'Hg', 576.959  ],
          'Hg-579' :[ 'Hg', 579.065  ],
          'Hg-611' :[ 'Hg', 610.8    ],
          'Hg-615' :[ 'Hg', 614.9475 ]

}

In [ ]:
HG_lines

{'Hg-404': ['Hg', 404.6563],
 'Hg-436': ['Hg', 435.8328],
 'Hg-543': ['Hg', 543.6],
 'Hg-546': ['Hg', 546.0735],
 'Hg-576': ['Hg', 576.959],
 'Hg-579': ['Hg', 579.065],
 'Hg-611': ['Hg', 610.8],
 'Hg-615': ['Hg', 614.9475]}

### Sentinal S2a

Information on the Sentinel spectral specificatoins can be found at:
https://sentinels.copernicus.eu/web/sentinel/user-guides/sentinel-2-msi/resolutions/spectral

Sentinal S2a Band Definitions.  `sentinal_s2a` defines the band name, center wavelength and
bandwidth in nanometers.  These are used to generate a class for each band that will hold
values that are computed once, and then used repeatedly.

In [ ]:
#| export
sentinal_s2a = {
# Band
#         Center
#                Width
  'b2' : [ 492.7, 65],
  'b3' : [ 559.8, 35],
  'b4' : [ 664.6, 30],
  'b8' : [ 832.8, 105],
  
  'b5' : [ 704.1, 14],
  'b6' : [ 740.5, 14],
  'b7' : [ 782.8, 19],
  '8a' : [ 864.7, 21],
}

#| hide
#### Sentinal_s2a Band 

In [ ]:
#| export
class Sentinal_Band:
  '''
  Defines a single Sentinel band data object. 
  '''
  header = f'band center_nm width_nm color w_low px_low w_high px_high '
  average = True
  mean    = False
  
  def __init__(self,  
               x:list,          # A Numpy array of wavelengths cooresponding to each pixel.
               name:str,        # The Sentinel official wavelength-band name.
               center_nm:float, # Center wavelength in nanometers.
               width_nm:float,  # Bandwidth in nanometers.
               color:str        # Desired color to use for any plots, or graphics.
              ) -> object:      # Class object returned.
    self.name      = name
    self.center_nm = center_nm
    self.width_nm  = width_nm
    self.color     = color
    self.w_low     = center_nm - width_nm/2
    self.w_high    = center_nm + width_nm/2
    self.px_low    = w_to_pixel( x, self.w_low )
    self.px_high   = w_to_pixel(x, self.w_high )
    self.pixels    = w_range_pixels(x, self.w_low,self.w_high)
    self.y_mean    = 0.0        # This gets updated.
  
  def __str__(self):
    return f'{self.name} {self.center_nm:6.2f} {self.width_nm:6.1f} {self.color} '\
           f'{self.w_low:6.1f} {self.px_low:4d} {self.w_high:6.1f} {self.px_high:4d}'
  
  def compute_stats(self, 
                  spectra     # Spectra to update y_mean from.
                 ) -> None:   # The stat values are updated in the class.
    '''
    Compute the Sentinel band stats for a given spectra if enabled. To enable or
    disable set Sentinel_Band.average = True | False, or Sentinel_Band.mean = True | False.
    '''
    if self.mean:
      self.y_mean    = spectra[ self.pixels ].mean()
    else:
      self.y_mean = 0.0
      
    if self.average:
      self.y_average = np.average( spectra[ self.pixels ] )
    else:
      self.y_average = 0.0  

In [ ]:
sa = {}
print(Sentinal_Band.header )
for k in sentinal_s2a.keys():
  sa[k] = Sentinal_Band( wavelengths, k, sentinal_s2a[k][0], sentinal_s2a[k][1], 'gray')
  print(sa[k].__str__() )

band center_nm width_nm color w_low px_low w_high px_high 
b2 492.70   65.0 gray  460.2  150  525.2  322
b3 559.80   35.0 gray  542.3  367  577.3  460
b4 664.60   30.0 gray  649.6  651  679.6  730
b8 832.80  105.0 gray  780.3  997  885.3 1274
b5 704.10   14.0 gray  697.1  777  711.1  814
b6 740.50   14.0 gray  733.5  873  747.5  910
b7 782.80   19.0 gray  773.3  978  792.3 1028
8a 864.70   21.0 gray  854.2 1192  875.2 1247


In [ ]:
sa['b3'].__str__()

'b3 559.80   35.0 gray  542.3  367  577.3  460'

In [ ]:
sa['b3'].center_nm

559.8

In [ ]:
band = sa['b3'].pixels
band

(array([368, 369, 370, 371, 372, 373, 374, 375, 376, 377, 378, 379, 380,
        381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 391, 392, 393,
        394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404, 405, 406,
        407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419,
        420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432,
        433, 434, 435, 436, 437, 438, 439, 440, 441, 442, 443, 444, 445,
        446, 447, 448, 449, 450, 451, 452, 453, 454, 455, 456, 457, 458,
        459, 460]),)

In [ ]:
test_spectra[ band ].mean()

7.798266129032258

In [ ]:
#| export
def sentinel_update( 
  spectra,           # Raw spectra to compute Sentinel band averages on.
  sentinel_bands     # Array of sentinel band classes.       
) -> None:           # sentinel_bands will contain .y_mean afterward.
  '''
  '''
  for s in sentinel_bands:
    sentinel_bands[ s ].compute_stats( spectra )

In [ ]:
%%timeit
sentinel_update( test_spectra, sa )

33.6 µs ± 399 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [ ]:
print('Band    y_mean   y_average')
for se in sa:
  sa[se].compute_stats( test_spectra )
  
  print(f'  {se}   {sa[se].y_mean:7.3f}  {sa[se].y_average:7.3f}')
  #print(y_avg)

Band    y_mean   y_average
  b2     0.000    4.463
  b3     0.000    7.798
  b4     0.000    2.543
  b8     0.000    0.003
  b5     0.000    0.322
  b6     0.000    0.000
  b7     0.000    0.000
  8a     0.000    0.000


In [ ]:
sentinel_compute( test_spectra, sa )
for band in sa:
  print(band, sa[ band ].y_mean)

b2 4.463212209302326
b3 7.798266129032258
b4 2.5434177215189875
b8 0.0030324909747293056
b5 0.3216216216216214
b6 0.0
b7 0.0
8a 0.0


### Plot helpers

#| hide
#### def gen_reference_lines()

Generate reference spectral line data suitable to plot with Bokeh.

In [ ]:
def gen_reference_lines(s,      # Spectra 
              lines,            # The reference lines.
             ) -> (list,list):  # Fraun X values, Fraun y values 
  '''
  Generate a series of spectral plot lines for reference.  It scales to the max y value 
  of the provided spectra `s`.  Sources can be Fraunhofer_lines, or HG_lines, or user 
  developed.
  '''
  fran_x = []; fran_y = []
  y0 = 0.0;    y1 = max(s)
  for i in lines:
    x = lines[i][1]
    fran_x.append( [ x,x ] )
    fran_y.append( [y0, y1] )
  return fran_x, fran_y

In [ ]:
spectra_file = "/mnt/s/2021-0717-HAB-Spec-Tampa-Bay-PIE/2021-0717-HAB-pie/165347/hab_spectra/2021-0717-165910-141598-spec.json"

test_spectra = [  
  ["/mnt/s/2021-0717-HAB-Spec-Tampa-Bay-PIE/2021-0717-HAB-pie/164443/hab_spectra/2021-0717-164700-525866-spec.json",
   "Gulf of Mexico west of Clearwater."
  ],
  
  ["/mnt/s/2021-0717-HAB-Spec-Tampa-Bay-PIE/2021-0717-HAB-pie/165347/hab_spectra/2021-0717-165910-141598-spec.json",
   "Old Tampa Bay. Very high 700nm peak"
  ],
  
  [ "/mnt/s/2021-1208-HAB-Lake-Parker/2021-1208-HAB-Lake-Parker/190829/hab_spectra/2021-1208-190953-177078-spec.json",
   "Lake Parker, Water." 
  ],
  
  [ "/mnt/s/2021-1208-HAB-Lake-Parker/2021-1208-HAB-Lake-Parker/175549/hab_spectra/2021-1208-175627-867603-spec.json",
   "Lake Parker, Sky."
  ]  
]

In [ ]:
spectra_number = 0
print(f'   File: {test_spectra[spectra_number][0]}')
print(f'Comment: {test_spectra[spectra_number][1]}')

   File: /mnt/s/2021-0717-HAB-Spec-Tampa-Bay-PIE/2021-0717-HAB-pie/164443/hab_spectra/2021-0717-164700-525866-spec.json
Comment: Gulf of Mexico west of Clearwater.


### Generate a plot of the Spectra.

In [ ]:
#spectra_file = '../sample_data/2023-0718/034901/hab_spectra/2023-0718-034915-344969-spec.json'
from bokeh.models import ColumnDataSource, Label, LabelSet, Range1d
ts = test_spectra[3]
spectra_file = ts[0]
screen_height = 300
screen_width  = 700

spectra = read_spectra( spectra_file )
spectra_w = figure(title= f"{ spectra_file.split('/')[-1]},    {ts[1]}",
                   x_axis_label='Wavelength (nm)', y_axis_label='Intensity (Counts)',
                  height=300, width=700)

fran_x, fran_y = gen_reference_lines( spectra, Fraunhofer_lines )

# Plot Sentinel bands.
sentinel_update( spectra, sa )
for se in sa:
  spectra_w.rect( x= sa[se].center_nm, y= sa[se].y_average, width=sa[se].width_nm, height= 2,
                 line_width=1, fill_color= wavelength_2_RGB_str( sa[se].center_nm, alpha=100 ) )
  
junk = spectra_w.multi_line(fran_x, fran_y, color='lightgray', legend_label="Fraunhofer Lines." )
junk = spectra_w.line(wavelengths, spectra, legend_label="Spectra", line_width=3)
label = Label( x=20, y=10, x_units='screen', y_units='screen', text= ts[1] )
#spectra_w.add_layout( label )
show(spectra_w)

#| hide
### HABSpec Navigation and positioning

#| hide
#### def nearest_rgb_image( fn, debug=False):

In [ ]:
#| export
#debug_nearest_rgb_image = False
def nearest_rgb_image( 
  fn,          # Json spectra file.
  debug=False  # True to debug internals.
) -> str:      # File name of the nearest RGB photo to this json spectra.
    """
    Returns the path/filename to the RGB photo closes in time to `fn`.  `fn` is the
    filename of a Json hyperspectral file.

    Parameters:
    -----------
    fn : str
      Full path/filename of a hyperspectral Json file.

    Returns:
    --------
    str
      A string path/filename of the closes RGB photo.

    Examples:
    ---------

    References:
    -----------
    """
    if debug:
      print(f'debug_nearest_rgb_image(fn): fn={fn}')
    fn_parts = fn.split('/')   # Split path by '/'
    if debug:
      print(f'debug_nearest_rgb_image(fn): fn_parts={fn_parts}')
    rgb_p = fn_parts                          # Make a copy to build the rgb path/filename in.
    if debug:
      print(f'debug_nearest_rgb_image(fn): rgb_p={rgb_p}')    
    rgb_p[-2] = 'hab_rgb'                     # Change the subdir to point to hab_rgb
    rgb_fn = rgb_p[-1].split('-')             # Split the filename by '-' to access parts.
    rgb_fn[-2] = '*'                          # Replace the microseconds with '*' wildcard.
    rgb_fn[-1] = 'rgb.jpg'                    # Change the file tail to rgb.jpg
    rgb_p[-1] = '-'.join(rgb_fn)              # Glue the name back together
    rgb_p2 = '/'.join(rgb_p)                  # Now glue the whole path back together
    rv = glob.glob(rgb_p2)[0]                 # Return the first entry incase there are more than 1.
    if debug:
      print(f'debug_nearest_rgb_image(fn): rv={rv}') 
    return rv


In [ ]:
print(
f'    Input Json spectra file: {test_spectra_file}\n'
f'        Nearest RGB File is: {nearest_rgb_image( test_spectra_file )}'  
)

    Input Json spectra file: ../sample_data/034901/hab_spectra/2023-0718-034912-259843-spec.json
        Nearest RGB File is: ../sample_data/034901/hab_rgb/2023-0718-034912-948434-rgb.jpg


####   def read_gps_file_to_df( ifn, debug=False):

In [ ]:
#| export
#debug_read_gps_file_to_df = False
def read_gps_file_to_df( ifn, debug=False):
    '''
    Read a GPS datafile into a dataframe and convert the HH:MM:SS
    to add an SOD column.

    Parameters:
    -----------
    ifn : str
        Input GPS datafile full path name

    Returns:
    --------
    Pandas Dataframe of the GPS data.
    '''
    gps_df = pd.read_csv(ifn, sep='\s+', comment='#')
    gps_df.sort_values(by=['HMS'], inplace=True)
    # Convert the ASCII HH:MM:SS from the GPS to seconds of the day (SOD) and
    # add an 'SOD' column to the gps dataframe.
    sod_lst = []
    for t in gps_df['HMS']:
      sod_lst.append( hms2sod(t))
    gps_df['SOD'] = sod_lst

    # Compute, and add 'Course' to dataframe
    course = []
    for i in  range(len(gps_df)-1):
      course.append ( get_bearing( 
          lat1=gps_df['Lat'].iloc[i], long1=gps_df['Lon'].iloc[i], 
          lat2=gps_df['Lat'].iloc[i+1], long2=gps_df['Lon'].iloc[i+1] 
          ) 
      )
    course.append( 0.0 )      # To make the same length
    gps_df['Course'] = course
    return gps_df

Test read_gps_file_to_df() function.  Read a datafile into a Pandas dataframe.

In [ ]:
Gps_File_Name = '/mnt/f/My Drive/Missions/2021-1208-HAB-Lake-Parker/2021-1208/2021-1208-175324-910850-mission-gps.txt'
Gps_File_Name = '../sample_data/034901/2023-0718-034805-411046-flightline-gps.txt'
gps_df = read_gps_file_to_df(Gps_File_Name, debug=False)
display(gps_df)

,Date,HMS,Lat,Lon,Elev,Speed,Nsat,Mode,Herr,Verr,Temp,SOD,Course
0,2023-07-18,03:49:01,28.209750,-82.363932,22.50,0.0,4,3,25.0,44.9,68.756,13741.0,19.402235
1,2023-07-18,03:49:02.100000,28.209751,-82.363932,22.50,0.0,4,3,25.0,44.9,68.756,13742.1,16.761456
2,2023-07-18,03:49:03.100000,28.209751,-82.363932,22.40,0.0,4,3,25.0,44.9,68.756,13743.1,30.312737
3,2023-07-18,03:49:04,28.209752,-82.363932,22.40,0.0,4,3,25.0,44.9,68.756,13744.0,60.193171
4,2023-07-18,03:49:05,28.209752,-82.363931,22.30,0.0,4,3,25.0,44.9,68.218,13745.0,-16.761454
5,2023-07-18,03:49:06.100000,28.209753,-82.363931,22.50,0.0,4,3,25.0,44.9,68.756,13746.1,46.024777
6,2023-07-18,03:49:07.100000,28.209754,-82.363930,22.40,0.0,4,3,25.0,44.9,68.756,13747.1,16.294989
7,2023-07-18,03:49:08.200000,28.209755,-82.363929,22.30,0.0,4,3,25.0,44.9,69.294,13748.2,138.464356
8,2023-07-18,03:49:09,28.209754,-82.363929,22.30,0.0,4,3,25.0,44.9,69.294,13749.0,41.535644
9,2023-07-18,03:49:10,28.209755,-82.363929,22.30,0.0,4,3,25.0,44.9,69.294,13750.0,33.467529


####   def compute_gps_positions(  gps, spec_df, debug=False):
See: https://numpy.org/doc/stable/reference/generated/numpy.interp.html

In [ ]:
#| export
#debug_compute_gps_positions = False 
def compute_gps_positions(  gps, spec_df, debug=False):
    '''
    Interpolates spectra positions from gps.

    Parameters:
    -----------
    gps : dataframe
        A dataframe of overlapping gps position data.

    spec_df : dataframe
        A dataframe of values vs time from the spectrometer.

    Returns:
    --------
    See: https://numpy.org/doc/stable/reference/generated/numpy.interp.html
    '''
    spec_df['Lat']    = np.interp(spec_df['sod'], gps['SOD'], gps['Lat'])
    spec_df['Lon']    = np.interp(spec_df['sod'], gps['SOD'], gps['Lon'])
    spec_df['Elev']   = np.interp(spec_df['sod'], gps['SOD'], gps['Elev'])
    spec_df['Course'] = np.interp(spec_df['sod'], gps['SOD'], gps['Course'])
    return spec_df

#### def get_bearing( lat1=0, lat2=0, long1=0, long2=0, debug=False):

In [ ]:
#| export
#debug_get_bearing = False
def get_bearing( lat1=0, lat2=0, long1=0, long2=0, debug=False):
    '''
    Comutes and returns the bearing between two lat/lon pairs.
    See:
        http://shorturl.at/atGHN
    '''
    brng = Geodesic.WGS84.Inverse(lat1, long1, lat2, long2)['azi1']
    return brng

In [ ]:
# Test get_bearing()
lst = [ [30,29, -75, -75],
        [29,30, -75, -75],
        [30,30, -76, -75],
        [30,30, -75, -76] ]
print('\nTesting get_bearing()')
for t in lst:
  v = get_bearing( lat1=t[0], lat2=t[1], long1=t[2], long2=t[3] )
  print( f'  {t}, {v:6.2f}')


Testing get_bearing()
  [30, 29, -75, -75], 180.00
  [29, 30, -75, -75],   0.00
  [30, 30, -76, -75],  89.75
  [30, 30, -75, -76], -89.75


#### def get_list_of_flight_lines( p, debug=False):

In [ ]:
#| export
#debug_get_list_of_flight_lines = False
def get_list_of_flight_lines( p, debug=False):
    '''Returns a list of flightline subdirs on path p.

    Parameters:
    -----------
    p : str
        Path name.

    Returns:
    --------
    list
        A list of full pathnames to individul flightlines.
    '''
    l = glob.glob(p+'/[0-9]*[!a-z]')
    return l

Test get_list_of_flight_lines().

In [ ]:
print('\nTesting: get_list_of_flight_lines(p)')
test_path_list = get_list_of_flight_lines( mission_folder )
display(test_path_list[0:5])


Testing: get_list_of_flight_lines(p)


[]

#### def get_list_of_hab_files( p, subdir='', ext='', debug=False):

In [ ]:
#| export
#debug_get_list_of_hab_files = False
def get_list_of_hab_files( p, subdir='', ext='', debug=False):
    '''Returns a list of hab files in p/subdir with the specified file extension.  
    
    Parameters:
    p : str
    subdir : str Default = ''
    ext : str Default = ''

    Returns:
    --------
    list
        A list of  full pathnames.
    '''
    gs = p+f'/{subdir}/*.'+ext
    js = glob.glob(gs)
    return js

Test get_list_of_hab_files().

In [ ]:
#mission_folder = '/mnt/s/2021-0717-HAB-Spec-Tampa-Bay-PIE/2021-0717-HAB-pie/'
print('\nTesting: get_list_of_hab_files()')
flight_line_list = get_list_of_flight_lines(mission_folder)
rv_get_list_of_hab_files = get_list_of_hab_files( flight_line_list[1], subdir='hab_spectra', ext='json')
display(rv_get_list_of_hab_files[0:5])

In [ ]:
print(f'{now_utc()}: HyperSpectral functions loaded.')

2023-0721 03:39:37: HyperSpectral functions loaded.


In [ ]:
rv_get_list_of_hab_files

In [ ]:
mission_folder, flight_line_list 

('../sample_data/', ['../sample_data/034901'])

In [ ]:
flight_line_list

['../sample_data/034901']

### Depreciated, to be removed.

In [ ]:
band_500_600[0][-1]

520

####   def get_fluorescence( x, y, fl_start=0, fl_stop=0, base_start=0, base_stop=0, debug=False ):

In [ ]:
#| export
#debug_get_fluorescence = False
def get_fluorescence( x, y, fl_start=0, fl_stop=0, base_start=0, base_stop=0, debug=False ):
    '''
    '''
    fl_sig = y[w_range_pixels(x, fl_start, fl_stop )].mean()         # Get the Fluor signal mean value
    by = y[w_range_pixels(x, base_start, base_start+1)].mean()
    ey = y[w_range_pixels(x, base_stop, base_stop+1)].mean()
    center_nm = (fl_stop - fl_start) / 2  + fl_start                                       # Compute center wavelength
    dydx = (ey - by)/(base_stop - base_start )
    sf = dydx * (center_nm - base_start)
    rv = fl_sig + sf
    if debug:
      print('get_fluorescence():', fl_sig, by, ey, center_nm, dydx, sf, rv)
    return rv

#### def get_fluorescence_700( x, y):

In [ ]:
#| export
def get_fluorescence_700( x, y):
    '''
    Returns the fluorescence value at 700nm.
    '''
    rv = get_fluorescence( x, y, fl_start=693, fl_stop=710, base_start=668, base_stop=740 )
    return rv

#### def get_fluorescence_683( x, y):

In [ ]:
#| export
def get_fluorescence_683( x, y):
    '''
    Returns the fluorescence value at 683nm.  683nm is Chlorophyll
    '''
    rv = get_fluorescence( x, y, fl_start=678, fl_stop=688, base_start=668, base_stop=740 )
    return rv